### Install Python SDK

In [ ]:
# Install Vertex AI LLM SDK
! pip install --user --upgrade google-cloud-aiplatform==1.47.0 langchain==0.1.14 langchain-google-vertexai==0.1.3 typing_extensions==4.9.0

# Dependencies required by Unstructured PDF loader
! sudo apt -y -qq install tesseract-ocr libtesseract-dev
! sudo apt-get -y -qq install poppler-utils
! pip install --user --upgrade unstructured==0.12.4 pdf2image==1.17.0 pytesseract==0.3.10 pdfminer.six==20221105
! pip install --user --upgrade pillow-heif==0.15.0 opencv-python==4.9.0.80 unstructured-inference==0.7.24 pikepdf==8.13.0 pypdf==4.0.1

# For Matching Engine integration dependencies (default embeddings)
! pip install --user --upgrade tensorflow_hub==0.16.1 tensorflow_text==2.15.0
! pip install sentence-transformers
! pip install -U langchain-community faiss-gpu
! pip install --upgrade --quiet  sentence_transformers > /dev/null
! pip install langchain_community
! pip install openai

libtesseract-dev is already the newest version (4.1.1-2.1build1).
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.
  Using cached pdfminer.six-20221105-py3-none-any.whl (5.6 MB)
  Attempting uninstall: pdfminer.six
    Found existing installation: pdfminer.six 20231228
    Uninstalling pdfminer.six-20231228:
      Successfully uninstalled pdfminer.six-20231228
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pdfplumber 0.11.0 requires pdfminer.six==20231228, but you have pdfminer-six 20221105 which is incompatible.


  Using cached pdfminer.six-20231228-py3-none-any.whl (5.6 MB)
  Attempting uninstall: pdfminer.six
    Found existing installation: pdfminer.six 20221105
    Uninstalling pdfminer.six-20221105:
      Successfully uninstalled pdfminer.six-20221105


# Import libraries

In [ ]:
import json
import textwrap

# Utils
import time
import uuid
from typing import List

import numpy as np
import vertexai

# Vertex AI
from google.cloud import aiplatform

print(f"Vertex AI SDK version: {aiplatform.__version__}")

# LangChain
import langchain

print(f"LangChain version: {langchain.__version__}")

from langchain.chains import RetrievalQA
from langchain.document_loaders import GCSDirectoryLoader
from langchain.embeddings import VertexAIEmbeddings
from langchain.llms import VertexAI
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Import custom Matching Engine packages
# from utils.matching_engine import MatchingEngine
# from utils.matching_engine_utils import MatchingEngineUtils
# Import custom Matching Engine packages
from langchain_google_vertexai import VertexAI , VertexAIEmbeddings , VectorSearchVectorStore
import faiss
from faiss import IndexFlatL2
import numpy as np
import spacy
from langchain_community.embeddings import HuggingFaceEmbeddings
import json
import pdfplumber
from langchain_community.vectorstores import FAISS
import os
from google.colab import files
import zipfile
from langchain.document_loaders import HuggingFaceDatasetLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
from transformers import AutoTokenizer, pipeline
from langchain import HuggingFacePipeline
from langchain.chains import RetrievalQA

Vertex AI SDK version: 1.47.0
LangChain version: 0.1.14


In [ ]:
model_name = 'all-mpnet-base-v2'
embeddings = HuggingFaceEmbeddings(model_name=model_name)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

# creating faiss db to store embeddings in offline mode

# Saving and loading

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#db.save_local("/content/drive/MyDrive/Capstone_project/faiss_index")

In [ ]:
new_db = FAISS.load_local("/content/drive/MyDrive/Capstone_project/faiss_index", embeddings,allow_dangerous_deserialization=True)
query = 'prolong alcohol intake impact human health?'

docs = new_db.similarity_search(query)

In [ ]:
docs

[Document(page_content='content: The Impact of Alcoholic Beverages on Human Health', metadata={'source': 'Title: The Impact of Alcoholic Beverages on Human Health  .pdf,page:2', 'chunk': 972}),
 Document(page_content='content: Al cohol affects human physiology been linked with an increase in risk of breast decreased health care utilisation, and increased either through years of cancer(9). Women are less likely to consume HIV risk behaviours due to lack of sobriety(14). alcohol than men; however, the use of alcohol Some countries have also found a four time consumption, acute intoxication, may have more implications for women than increased risk of multimorbidity in individuals or dependence(5). It has been men with respect to physical illnesses and more who drink alcohol(15). linked with approximately 230 severe cognitive and motor impairment with ICD-10 (International Classification a much lower alcohol exposure as compared to men(10). Beyond the direct consequences on health of the o

In [ ]:
new_db.similarity_search_with_score(query)

[(Document(page_content='content: The Impact of Alcoholic Beverages on Human Health', metadata={'source': 'Title: The Impact of Alcoholic Beverages on Human Health  .pdf,page:2', 'chunk': 972}),
  0.58215624),
 (Document(page_content='content: Al cohol affects human physiology been linked with an increase in risk of breast decreased health care utilisation, and increased either through years of cancer(9). Women are less likely to consume HIV risk behaviours due to lack of sobriety(14). alcohol than men; however, the use of alcohol Some countries have also found a four time consumption, acute intoxication, may have more implications for women than increased risk of multimorbidity in individuals or dependence(5). It has been men with respect to physical illnesses and more who drink alcohol(15). linked with approximately 230 severe cognitive and motor impairment with ICD-10 (International Classification a much lower alcohol exposure as compared to men(10). Beyond the direct consequences o

# Designing custom retreiver to generate_summary

# Model gpt2

In [ ]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [ ]:
# converting it into list of string to generate summary
docs1=format_docs(docs)

In [ ]:
docs1

"content: The Impact of Alcoholic Beverages on Human Health\n\ncontent: Al cohol affects human physiology been linked with an increase in risk of breast decreased health care utilisation, and increased either through years of cancer(9). Women are less likely to consume HIV risk behaviours due to lack of sobriety(14). alcohol than men; however, the use of alcohol Some countries have also found a four time consumption, acute intoxication, may have more implications for women than increased risk of multimorbidity in individuals or dependence(5). It has been men with respect to physical illnesses and more who drink alcohol(15). linked with approximately 230 severe cognitive and motor impairment with ICD-10 (International Classification a much lower alcohol exposure as compared to men(10). Beyond the direct consequences on health of the of Diseases, 10th edition) diseases, drinker, the chronic use of alcohol is responsible including 40 diseases that would not for a significant societal impa

In [ ]:
from transformers import pipeline
model_name="gpt2"
chat_pipeline=pipeline("text-generation",model=model_name)

from transformers import GPT2LMHeadModel, GPT2Tokenizer
from langchain.prompts import PromptTemplate

model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:

chat_pipeline(docs1,max_length=1000)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': "content: The Impact of Alcoholic Beverages on Human Health\n\ncontent: Al cohol affects human physiology been linked with an increase in risk of breast decreased health care utilisation, and increased either through years of cancer(9). Women are less likely to consume HIV risk behaviours due to lack of sobriety(14). alcohol than men; however, the use of alcohol Some countries have also found a four time consumption, acute intoxication, may have more implications for women than increased risk of multimorbidity in individuals or dependence(5). It has been men with respect to physical illnesses and more who drink alcohol(15). linked with approximately 230 severe cognitive and motor impairment with ICD-10 (International Classification a much lower alcohol exposure as compared to men(10). Beyond the direct consequences on health of the of Diseases, 10th edition) diseases, drinker, the chronic use of alcohol is responsible including 40 diseases that would not for a signi

# Text generator

In [ ]:
def generate_text(prompt, max_length=1000):
  input_ids = tokenizer.encode(prompt, return_tensors="pt")
  output = model.generate(input_ids, max_length=max_length, num_return_sequences=1, temperature=0.7)
  generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
  return generated_text

In [ ]:
generate_text(docs1)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


"content: The Impact of Alcoholic Beverages on Human Health\n\ncontent: Al cohol affects human physiology been linked with an increase in risk of breast decreased health care utilisation, and increased either through years of cancer(9). Women are less likely to consume HIV risk behaviours due to lack of sobriety(14). alcohol than men; however, the use of alcohol Some countries have also found a four time consumption, acute intoxication, may have more implications for women than increased risk of multimorbidity in individuals or dependence(5). It has been men with respect to physical illnesses and more who drink alcohol(15). linked with approximately 230 severe cognitive and motor impairment with ICD-10 (International Classification a much lower alcohol exposure as compared to men(10). Beyond the direct consequences on health of the of Diseases, 10th edition) diseases, drinker, the chronic use of alcohol is responsible including 40 diseases that would not for a significant societal impa

# Text summarizer

In [ ]:

def summarize_text(text, max_length=1000):
  input_ids = tokenizer.encode(text, return_tensors="pt", max_length=1024, truncation=True)
  output = model.generate(input_ids, max_length=max_length, num_return_sequences=1, temperature=0.2, early_stopping=True)
  summarized_text = tokenizer.decode(output[0], skip_special_tokens=True)
  return summarize_text

In [ ]:
len(docs)

4

In [ ]:
from transformers import pipeline

def summarize_text(text, max_length=1000):
    """Summarize input text using a pre-trained GPT-2 model."""
    summarization_pipeline = pipeline("summarization", model="gpt2")
    summary = summarization_pipeline(text, max_length=max_length, min_length=50, do_sample=True)[0]['summary_text']
    return summary

# Example usage:
input_text = docs1
summary = summarize_text(input_text)
print("Summary:", summary)

The model 'GPT2LMHeadModel' is not supported for summarization. Supported models are ['BartForConditionalGeneration', 'BigBirdPegasusForConditionalGeneration', 'BlenderbotForConditionalGeneration', 'BlenderbotSmallForConditionalGeneration', 'EncoderDecoderModel', 'FSMTForConditionalGeneration', 'GPTSanJapaneseForConditionalGeneration', 'LEDForConditionalGeneration', 'LongT5ForConditionalGeneration', 'M2M100ForConditionalGeneration', 'MarianMTModel', 'MBartForConditionalGeneration', 'MT5ForConditionalGeneration', 'MvpForConditionalGeneration', 'NllbMoeForConditionalGeneration', 'PegasusForConditionalGeneration', 'PegasusXForConditionalGeneration', 'PLBartForConditionalGeneration', 'ProphetNetForConditionalGeneration', 'SeamlessM4TForTextToText', 'SeamlessM4Tv2ForTextToText', 'SwitchTransformersForConditionalGeneration', 'T5ForConditionalGeneration', 'UMT5ForConditionalGeneration', 'XLMProphetNetForConditionalGeneration'].
Your max_length is set to 1000, but your input_length is only 580

Summary: content: The Impact of Alcoholic Beverages on Human Health

content: Al cohol affects human physiology been linked with an increase in risk of breast decreased health care utilisation, and increased either through years of cancer(9). Women are less likely to consume HIV risk behaviours due to lack of sobriety(14). alcohol than men; however, the use of alcohol Some countries have also found a four time consumption, acute intoxication, may have more implications for women than increased risk of multimorbidity in individuals or dependence(5). It has been men with respect to physical illnesses and more who drink alcohol(15). linked with approximately 230 severe cognitive and motor impairment with ICD-10 (International Classification a much lower alcohol exposure as compared to men(10). Beyond the direct consequences on health of the of Diseases, 10th edition) diseases, drinker, the chronic use of alcohol is responsible including 40 diseases that would not for a significant societa

In [ ]:
# Create a retriever object from the 'db' using the 'as_retriever' method.
# This retriever is likely used for retrieving data or documents from the database.
retriever = new_db.as_retriever()
docs = retriever.invoke('prolong alcohol intake impact human health?')
docs

[Document(page_content='content: The Impact of Alcoholic Beverages on Human Health', metadata={'source': 'Title: The Impact of Alcoholic Beverages on Human Health  .pdf,page:2', 'chunk': 972}),
 Document(page_content='content: Al cohol affects human physiology been linked with an increase in risk of breast decreased health care utilisation, and increased either through years of cancer(9). Women are less likely to consume HIV risk behaviours due to lack of sobriety(14). alcohol than men; however, the use of alcohol Some countries have also found a four time consumption, acute intoxication, may have more implications for women than increased risk of multimorbidity in individuals or dependence(5). It has been men with respect to physical illnesses and more who drink alcohol(15). linked with approximately 230 severe cognitive and motor impairment with ICD-10 (International Classification a much lower alcohol exposure as compared to men(10). Beyond the direct consequences on health of the o

In [ ]:
docs = retriever.get_relevant_documents("What is prolong alcohol intake impact human health?")

context_summ=' '.join([str(i.page_content) for i in docs])


In [ ]:
context_summ

"., 2022). Assessing the level of alcohol consumption and (Ansari, Stock, & Mills, 2013); (Ukwayi, Lucy, understanding the detrimental effects it has on Chibuzo, & Undelikwo, 2013); (Tembo, Burns, & human health. Kalembo, 2017); (Dlamini, Rugbeer, Naidoo, Metso, & Moodley, 2012) Outcomes in the first objective that is shown in the table above are that alcohol consumption is having a detrimental effect on some students and families, leading to a potential increase in dropouts and poor academic performance [20]. It is very important to examine the level of alcohol consumption or intake and to understand its harmful effects on human health as mentioned in Table 1. The consumption of alcohol harms human health throughout one's life. Previous studies have linked alcohol consumption to numerous diseases including cancer, pancreatitis, liver cirrhosis, tuberculosis, pneumonia, diabetes mellitus, alcohol use disorder, cancers, psychiatric conditions, and injury [21] content: The Impact of Alco

In [ ]:
docs = retriever.get_relevant_documents("What is e-cigarrette?")

context_summ=' '.join([str(i.page_content) for i in docs])


In [ ]:
context_summ

'. The advent of e-cigarettes as a device to self-administer addictive agents such as cocaine and synthetic cathinones may bring about additional adverse health effects associated with their concurrent use. This review aims to briefly describe e-cigarettes and their different generations, and their co-use with other addictive drugs as well as the use of the device as a tool to self-administer addictive drugs, such as cocaine, etc. content: Electronic cigarettes (e-cigarettes) are devices that allow the user to inhale nicotine in a vapor, and are primarily marketed as a means of quitting smoking and a less harmful replacement for traditional cigarette smoking. However, further research is needed to determine if vaping nicotine via e-cigarettes can be effective. Conversely, nicotine has been considered a gateway drug to alcohol and other addictive drugs and e-cigarettes containing nicotine may have the same effects. Previous reports have shown that e-cigarette use may open the gate for t